In [ ]:
stock_code = "601318"
url = f"https://stockpage.10jqka.com.cn/ajax/code/{stock_code}/type/news/"
headers = {
            'Referer': f"https://stockpage.10jqka.com.cn/{stock_code}/news/",
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
            'X-Requested-With': 'XMLHttpRequest',
        }
referer = f"https://stockpage.10jqka.com.cn/{stock_code}/news/"
response = requests.get(
            url,
            headers=headers,
            timeout=10
        )
response.raise_for_status()

In [32]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def parse_report_links(html_content):
    """先截取相关研报部分，再解析HTML内容提取相关研报链接和标题"""
    report_list = []
    
    # 先使用BeautifulSoup解析整个HTML
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 查找包含"相关研报"的块
    report_section = soup.find('div', id='report')
    
    # 如果找不到相关研报块，直接返回空列表
    if not report_section:
        return report_list
    
    # 从相关研报块中提取所有的dl元素
    report_items = report_section.find_all('dl')
    
    for item in report_items:
        title_link = item.find('a', class_='client')
        date_span = item.find('span', class_='date')
        
        if title_link and date_span:
            report = {
                'type': '相关研报',
                'publish_date': date_span.text.strip(),
                'title': title_link.get('title'),
                'href': title_link.get('href'),
            }
            report_list.append(report)
    
    return report_list

def parse_hot_news(html_content):
    """解析HTML内容,提取热点新闻链接、标题、发布日期和内容"""
    hot_news_list = []
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 查找id为'news'的部分
    news_section = soup.find('div', id='news')
    
    if news_section:
        # 解析第一种格式的新闻(dl格式)
        news_items = news_section.find_all('dl')
        for item in news_items:
            title_link = item.find('a', class_='client')
            date_span = item.find('span', class_='date')
            summary_dd = item.find('dd', class_='hot_preview')
            
            if title_link and date_span and summary_dd:
                summary = summary_dd.find('p').text.strip()
                hot_news = {
                    'type': '热点新闻',
                    'publish_date': date_span.text.strip('[]'),
                    'title': title_link.get('title'),
                    'href': title_link.get('href'),
                    'summary': summary,
                }
                hot_news_list.append(hot_news)
        
        # 解析第二种格式的新闻(ul>li格式)
        ul_lists = news_section.find_all('ul', class_='news_lists')
        for ul in ul_lists:
            li_items = ul.find_all('li')
            for li in li_items:
                a_tag = li.find('a', class_='client')
                date_span = li.find('span')
                
                if a_tag and date_span:
                    # 从a标签中提取标题和链接
                    title = a_tag.get('title')
                    href = a_tag.get('href')
                    
                    # 日期格式转换(07/18 -> 2025-07-18)
                    month_day = date_span.text.strip()
                    current_year = datetime.now().year
                    publish_date = f"{current_year}-{month_day.replace('/', '-')}"
                    
                    # 内容就是a标签内的文本(去除日期部分)
                    content = a_tag.text.replace(date_span.text, '').strip()
                    
                    hot_news = {
                        'type': '热点新闻',
                        'publish_date': publish_date,
                        'title': title,
                        'href': href,
                        'summary': content,
                    }
                    hot_news_list.append(hot_news)
    
    return hot_news_list
def parse_announcements(html_content):
    """解析HTML内容,提取公司公告链接、标题、发布日期"""
    announcement_list = []
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # 查找id为'pubs'的部分
    announcements_section = soup.find('div', id='pubs')
    
    if announcements_section:
        # 从'pubs'部分中提取公告列表
        announcement_items = announcements_section.find_all('li')
        for item in announcement_items:
            a_tag = item.find('a', class_='client')
            date_span = item.find('span')
            
            if a_tag and date_span:
                announcement = {
                    'type': '公司公告',
                    'publish_date': date_span.text.strip(),
                    'title': a_tag.get('title'),
                    'href': a_tag.get('href')
                }
                announcement_list.append(announcement)
    
    return announcement_list
html_content = response.text
    
if html_content:
    # news_list = parse_news_links(html_content)
    news_list = parse_hot_news(html_content)
    report_list = parse_report_links(html_content)
    announcements_list = parse_announcements(html_content)
    
    all_items = news_list + announcements_list + report_list
    print(pd.DataFrame(all_items))

    type publish_date                                              title  \
0   热点新闻   2025-07-19             中国平安：7月18日获融资买入2.29亿元，占当日流入资金比例为20.62%   
1   热点新闻   2025-07-19                       险资跑步进入大A超2000亿元，部分已进入了这些行业股票   
2   热点新闻   2025-07-18                    配置高股息权益资产构建OCI账户 重塑险资投资逻辑优化资金结构   
3   热点新闻   2025-07-18                            中国平安：2025年度长期服务计划暂无实施进展   
4   热点新闻   2025-07-18                                中国平安(7月18日)出现1笔大宗交易   
5   热点新闻   2025-07-18                 内险股涨幅居前 险资长期考核指标落地 南向通扩容有助于缓解资产端压力   
6   热点新闻   2025-07-18                        上市险企上半年保费收入稳健增长 新华保险同比增幅超两成   
7   热点新闻   2025-07-18                         险资首次股权投资海上风电，平安为何坚定选择新能源赛道   
8   热点新闻   2025-07-18                              中国平安：7月17日获融资买入1.21亿元   
9   热点新闻   2025-07-17                      韩占武会见中国平安党委副书记、总经理兼联席首席执行官谢永林   
10  热点新闻   2025-07-17                    中国平安：把青少年“保险科技助农”教育公益课堂搬到“田间地头”   
11  热点新闻   2025-07-17                        太保、新华、众安上半年保费收入公布，最高同比增长23%   
12  热点新闻   2

In [50]:
import requests

url = 'https://stock.10jqka.com.cn/hks/20250718/c669728719.shtml'
headers = {
    'referer': 'https://news.10jqka.com.cn/',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers)
response.encoding = response.apparent_encoding
html_content = response.text
# 使用 BeautifulSoup 解析 HTML 并提取纯文本
soup = BeautifulSoup(html_content, 'html.parser')

content_div = soup.find('div', id='contentApp')
if content_div:
    # 提取纯文本
    text_content = content_div.get_text(strip=True)
    print(text_content)
else:
    print("未找到指定的div内容")    
    plain_text = soup.get_text()
    cleaned_text = "\n".join(line.strip() for line in plain_text.split("\n"))
    print(cleaned_text)

2025年7月1日，中国平安（601318）与中广核风电有限公司（以下简称“中广核风电”）就中广核汕尾甲子、惠州港口海上风电项目的深入合作在广东完成签约。据悉，前期中国平安以37.26亿元成功中标中广核汕尾新能源有限公司和中广核新能源（惠州）有限公司两个公司的部分股权，底层资产为汕尾甲子、惠州港口2个海上风电电站，合计装机容量1.9GW，分别为粤东地区和粤港澳大湾区首个百万千瓦级海上风电项目。项目投运至今运行稳定，预期收益良好。上述项目由中国平安人寿保险股份有限公司（平安人寿）直接投资，平安信托有限责任公司（平安信托）提供咨询服务。上述项目将助力中广核风电形成存量资产和新增投资的良性循环，支持广东省能源结构改善升级，切实提高服务实体经济质效。同时，上述项目也是国内保险资金首次以平层形式直接股权投资海上风电项目，是中国平安积极贯彻落实中央金融工作会议精神，扎实做好绿色金融大文章，以实际行动服务国家能源安全战略的有力体现。新能源与险资的“双向奔赴”2024年9月印发的《国务院关于加强监管防范风险推动保险业高质量发展的若干意见》提出，发挥保险资金长期投资优势，培育真正的耐心资本，推动资金、资本、资产良性循环。2024年12月召开的中央经济工作会议指出，必须统筹好培育新动能和更新旧动能的关系，因地制宜发展新质生产力。能源变革是生产力进步的战略先导，新能源是新质生产力的重要构成。在新的形势下，各大能源企业都制定了超常规的发展目标，预计“十五五”内能源总投资规模将超过5万亿元。新能源电站资产将进一步向央企、地方能源集团集中，集中度将持续上升。相关预测显示，2025年末，“五大六小”电力央企的风光装机将接近1000GW，占比超50%；地方能源集团风光装机将超300GW，占比接近20%。但目前央国企和地方能源集团均受到各自不同的约束，央企的主要限制是资产负债率红线。2025年，国务院国资委要求央企“一利五率”指标“一增一稳四提升”，其中“一稳”即资产负债率要稳定在65%左右，目前，各大发电央企的资产负债率基本均接近该红线。对于地方能源集团，其实际控制人——地方政府目前普遍受到地方债务的影响，出资压力较大。通过政策引导和市场机制，将短期资金转化为长期资本，蓄势待发的险资也开始了对新能源电站建设的积极探索。保险资金的自信与勇气，源自其自身与新能源高度匹配的独特优势。平安信托，作为平安集